# 4개의 노드

4개의 노드로 구성되어 있는 파이프라인을 작성합니다.<br>
총 4개의 케이스를 작성하여 아래 그림과 같은 그래프 구성을 이루는 파이프라인을 구현합니다.

<img src='node_example_image/node4.png'></img>

In [1]:
import kfp
from kfp import components, dsl
from kfp.components import func_to_container_op

## A -> (B, C) -> D

3개 노드로 구성한 파이프라인 예제를 참고하여 4개의 노드를 구성합니다. <br>
출력과 입력이 두개로 구성되어 있는 노드A,D를 유의하여 작성합니다.<br>
> nodeA : B, C에 입력할 출력을 필수적으로 정의해야 합니다. <br>
> nodeD : B, C에서 출력한 값을 입력하기 위한 데이터 형태를 확실히 알아야 합니다.

In [2]:
from typing import NamedTuple
@func_to_container_op
def node_A() -> NamedTuple('Outputs', [('first', str), ('second', str)]):
    task_A = 'A'
    print(task_A)
    return (task_A, task_A)

@func_to_container_op
def node_B(B: str) -> str:
    task_B = f'{B} -> B'
    print(task_B)
    return task_B

@func_to_container_op
def node_C(C: str):
    task_C = f'{C} -> C'
    print(task_C)
    return task_C

3개의 노드로 구성되어진 예제 코드를 그대로 작성한 후 마지막에 두개의 노드의 출력을 입력으로 받는 노드D를 정의합니다.<br>
노드D는 3개 노드 예제를 참고합니다.

In [3]:
@func_to_container_op
def node_D(D1: str, D2:str):
    task_D = f'{(D1, D2)} -> D'
    print(task_D)
    return task_D

4개의 노드 출력, 입력을 연결하여 파이프라인을 완성합니다. 이때 노드 3개 파이프라인의 확장인 것을 확인할 수 있습니다.

In [4]:
def connect_example_pipeline():
    node_A_task = node_A()
    node_B_task = node_B(node_A_task.outputs['first'])
    node_C_task = node_C(node_A_task.outputs['second'])
    node_D_task = node_D(node_B_task.output, node_C_task.output)

### kubeflow pipeline 결과 화면

<img src='node_example_image/node4_1.png' width='850px'></img>

## A -> B, A -> C, A -> D

노드(컴포넌트) 구성 시 3개의 출력을 갖는 노드A를 유의해야 할 것 같지만 2개 이상의 출력은 보통 다른 데이터 type을 사용합니다. <br>
기존 3개의 노드로 구성되어 있는 파이프라인은 NamedTuple 패키지 사용 예제를 보여주기 위한 파이프라인을 구성하였습니다. 하지만 같은 값의 출력을 tuple 형태로 구성하였기 때문에 결국은 하나의 출력과 동일하다고 볼 수 있습니다.<br>
지금까지의 예제로 입력과 출력 방식을 확인하였기 때문에 현재 케이스에서는 기본적으로 구성한 컴포넌트를 파이프라인에서 3개의 노드로 연결하도록 코드를 작성합니다.
> NamedTuple은 서로 다른 형태의 2개의 값을 출력으로 가질 때 주로 사용합니다.<br>
> 간단한 인수가 아닌 더 큰 형태의 데이터는 list,InputPath, OutputPath를 사용합니다. <br>
> 본 예제에서는 파이프라인의 워크플로우 확인을 목적으로 하기때문에 NamedTuple까지만 사용합니다.

In [ ]:
@func_to_container_op
def node_A() -> str:
    task_A = 'A'
    print(task_A)
    return task_A

@func_to_container_op
def node_B(B: str) -> str:
    task_B = f'{B} -> B'
    print(task_B)
    return task_B

@func_to_container_op
def node_C(C: str) -> str:
    task_C = f'{C} -> C'
    print(task_C)
    return task_C

@func_to_container_op
def node_D(D: str) -> str:
    task_D = f'{D} -> D'
    print(task_D)
    return task_D

4개의 노드를 간단하게 정의합니다. 각 노드는 입력과 출력만 정확히 알고 있다면 파이프라인을 작성하는 단계에서 워크플로우를 파이프라인 작성자가 정의한다고 볼 수 있습니다. 노드A의 출력을 나머지 세노드가 입력으로 받도록 파이프라인을 작성합니다.

In [5]:
def connect_example_pipeline():
    node_A_task = node_A()
    node_B_task = node_B(node_A_task.output)
    node_C_task = node_C(node_A_task.output)
    node_D_task = node_D(node_A_task.output)

### kubeflow pipeline 결과 화면

<img src='node_example_image/node4_2.png' width='850px'></img>

#### 파이프라인 구성에 따라 kubeflow pipeline에서 워크플로우가 작성되기 때문에 입력과 출력의 개수, 형태를 정확히 인지해야 합니다. 지금부터는 최소한의 입력과 출력 인수로 파이프라인을 작성하는 것을 목표로 합니다.

## A -> D, B -> D, C -> D

기존 노드B,C만 입력이 없는 노드로 수정합니다.

In [6]:
@func_to_container_op
def node_B() -> str:
    task_B = f' B'
    print(task_B)
    return task_B

@func_to_container_op
def node_C() -> str:
    task_C = 'C'
    print(task_C)
    return task_C

이때, 노드D의 입력은 3개로 다시 작성해야 합니다. <br>만약 수정 없이 아래와 같이 파이프라인을 작성한다면 예제 그래프가 아닌 각각의 워크플로우를 확인할 수 있습니다.

In [7]:
def connect_example_pipeline():
    node_A_task = node_A()
    node_B_task = node_B()
    node_C_task = node_C()
    node_D_task = node_D(node_A_task.output)
    node_D_task = node_D(node_B_task.output)
    node_D_task = node_D(node_C_task.output)

<img src='node_example_image/node4_3.png' width='850px'></img>

때문에, 노드D도 3개의 입력을 받도록 수정한 후 세 노드의 출력을 한번에 받도록 파이프라인을 작성합니다.

In [8]:
@func_to_container_op
def node_D(D1: str, D2: str, D3: str) -> str:
    task_D = f'{(D1, D2, D3)} -> D'
    print(task_D)
    return task_D

def connect_example_pipeline():
    node_A_task = node_A()
    node_B_task = node_B()
    node_C_task = node_C()
    node_D_task = node_D(node_A_task.output, node_B_task.output, node_C_task.output)

### kubeflow pipeline 결과 화면

<img src='node_example_image/node4_4.png' width='850px'></img>

## A -> (B, C) -> D, A -> D

위의 예제를 통해 해당 예제는 간단한 노드 수정과 파이프라인 연결을 다르게 하여 작성이 가능한 것을 확인할 수 있습니다.
먼저 노드B,C가 노드A의 출력을 입력받을 수 있도록 수정합니다.

In [10]:
@func_to_container_op
def node_B(B: str) -> str:
    task_B = f'{B} -> B'
    print(task_B)
    return task_B

@func_to_container_op
def node_C(C: str) -> str:
    task_C = f'{C} -> C'
    print(task_C)
    return task_C

파이프라인도 노드B,C 연결만을 추가하여 작성합니다.

In [11]:
def connect_example_pipeline():
    node_A_task = node_A()
    node_B_task = node_B(node_A_task.output)
    node_C_task = node_C(node_A_task.output)
    node_D_task = node_D(node_A_task.output, node_B_task.output, node_C_task.output)

### kubeflow pipeline 결과 화면

<img src='node_example_image/node4_5.png' width='850px'></img>